In [1]:
library(survival)
library(ggplot2)
library(dplyr)
library(reshape2)
library(stringr)

Warning message:
“package ‘survival’ was built under R version 4.0.5”
Warning message:
“package ‘ggplot2’ was built under R version 4.0.5”
Warning message:
“package ‘dplyr’ was built under R version 4.0.5”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘reshape2’ was built under R version 4.0.5”
Warning message:
“package ‘stringr’ was built under R version 4.0.5”


In [2]:
dir_fig = 'R_clogit/for_publish_fin/'

file_dir = 'combined_tables_missing/'

#use_gw_cov = FALSE
use_gw_cov = TRUE
gw_suf = ''
gw_var_base = ''
#plat_var_suffs = c('_lr', '_lr_il_ilgt', '_il')
plat_var_suffs = c('_lr_il_ilgt', '_il')
if (use_gw_cov) {
    gw_suf = '_useGWB'
    gw_var_base = 'count_genomewide'
}

In [3]:
### clean features
sub_fts = c('intergenic',
            'LOWPLI_intron', 'LOWPLI_cds + LOWPLI_utr', 
            'PLIp9_intron', 'PLIp9_cds + PLIp9_utr', 
            'DEV_BR_GENE_10_intron', 'DEV_BR_GENE_10_cds + DEV_BR_GENE_10_utr')
sub_fts_col_names = c('intergenic', 
                  'lowpli_genes_intronic', 'lowpli_genes_exonic', 
                  'pli_genes_intronic', 'pli_genes_exonic', 
                  'fetal_brain_genes_intronic', 'fetal_brain_genes_exonic')

In [4]:
#prs_mode = 'sbayesr'
prs_mode = 'prscs'
#file_suf = 'SQ70_SAMPLES_AD2_SAMPLES'
file_suf = 'SQ20_SAMPLES_AD2_SAMPLES'
#file_suf = 'SQ30_SAMPLES_AD2_SAMPLES'
#file_suf = 'SQ40_SAMPLES_AD2_SAMPLES'
#file_suf = 'SQ50_SAMPLES_AD2_SAMPLES'
#file_suf = 'SQ60_SAMPLES_AD2_SAMPLES'
#file_suf = 'SQ70_SAMPLES_AD2_SAMPLES_HP12'
#file_suf = 'SQ70_SAMPLES_AD2_SAMPLES_NCOM'
#file_suf = 'SQ50_SAMPLES_AD2_SAMPLES'
file_suf_trmissing = '25'
#file_suf_trmissing = '20'
#file_suf_trmissing = '15'
if (prs_mode == 'sbayesr') {
    file_main = paste0(file_dir, 'combined_svs_trs_snvs_prs_pc_sbayesr_', file_suf, '_DELINS_GW_plat', '_TRmissingness', file_suf_trmissing, '_fltHomPol_famGT.csv')
} else if (prs_mode == 'prscs') {
    file_main = paste0(file_dir, 'combined_svs_trs_snvs_prs_pc_prscs_', file_suf, '_DELINS_GW_plat', '_TRmissingness', file_suf_trmissing, '_fltHomPol_famGT.csv')}

df_main <- read.table(file_main, header = TRUE, sep = ",")
#df_main['count_PLIp9_nonLOF_DEV_BR_10_cds'] = df_main[,'count_PLIp9_DEV_BR_GENE_10_cds'] - df_main[,'count_PLIp9_LOF_DEV_BR_10_cds']
df_main[,'Platform'] <- as.factor(df_main[,'Platform'])
df_main[,'sex'] <- as.factor(df_main[,'sex'])
dim(df_main)
df_main

[1]  243 5571

fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,⋯,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_del,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_ins,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q_del,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q_ins,PRS_ASD_Z,PRS_SCZ_Z,PRS_EDU_Z,count_dnlof_inhlof
<chr>,<chr>,<chr>,<fct>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
F0001,REACH000001,ASD,Male,1,trio,1,1,0.008627126,-0.005607061,⋯,0,0,0,0,0,0,-0.971701072,-0.510485305,0.82162924,0
F0026,REACH000026,ASD,Male,1,trio,1,0,0.095363540,-0.045229620,⋯,0,0,0,0,0,0,0.303644763,0.817056455,0.06312390,1
F0058,REACH000058,ASD,Male,1,trio,1,1,0.002421759,-0.009216429,⋯,0,0,0,0,0,0,0.595643472,-0.305436874,1.94303491,2
F0065,REACH000065,ASD,Male,1,trio,1,0,0.252773800,0.107932900,⋯,0,0,0,0,0,0,1.166191828,0.943709479,-2.78896425,1
F0078,REACH000086,ASD,Male,1,trio,2,0,0.044785250,0.038064880,⋯,0,0,0,0,0,0,0.703902177,-0.152554399,0.06482359,0
F0078,REACH000089,ASD,Female,1,trio,2,0,0.049879690,0.047204300,⋯,0,0,0,0,0,0,0.384605619,0.209070495,0.16440487,0
F0081,REACH000092,ASD,Female,1,trio,1,1,-0.028510680,-0.001001894,⋯,0,0,0,0,0,0,-0.898010204,-0.994547648,0.53088874,1
F0081,REACH000095,CON,Female,1,trio,1,1,-0.030440730,-0.002736118,⋯,0,0,0,0,0,0,-1.036073363,-1.487948561,0.29231934,1
F0083,REACH000097,ASD,Male,1,trio,1,0,0.095793110,0.052887850,⋯,0,0,0,0,0,0,0.817923575,-0.003587407,-0.08546761,1


In [5]:
df_main_ont = df_main %>% filter(Platform == 'ONT')
df_main_pb = df_main %>% filter(Platform == 'PB')
print(dim(df_main_ont))
print(dim(df_main_pb))

[1]  101 5571
[1]  142 5571


In [6]:
zs_thr = 3
supp_thr = 2
q_suf = '_Q'

gw_var_sv = 'count_genomewide'
gw_var_tr = paste0('count_ZS', zs_thr, '_genomewide_SUPP', supp_thr, '_Q')

cols_out = c('fid','iid','Phenotype','sex','MEAN_COVERAGE','Platform','rel','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',
             'count_dnlof_inhlof','PRS_ASD_Z', gw_var_sv, gw_var_tr)

df_out = df_main[cols_out]

for (i_ft in 1:length(sub_fts)) {
    ft = sub_fts[i_ft]
    col_out_base = sub_fts_col_names[i_ft]
    ft_tmp = unlist(strsplit(ft, split='+', fixed=TRUE)) # split vars with +
    ft_tmp = str_trim(ft_tmp) # trim spaces
    svs_tmp = paste0('count_', ft_tmp) # add count_
    trs_tmp = paste0('count_ZS', zs_thr, '_', ft_tmp, '_SUPP', supp_thr, q_suf) # add count_
    
    col_out = paste0('svs_', col_out_base)
    df_out[col_out] = 0
    for (ft_ in svs_tmp) {
        for (plat_var_suff in plat_var_suffs) { # add burden detected from each platform
            ft_plat = paste0(ft_, plat_var_suff)
            df_out[col_out] = df_out[col_out] + df_main[ft_plat]
        }
    }

    col_out = paste0('trs_', col_out_base)
    df_out[col_out] = 0
    for (ft_ in trs_tmp) {
        df_out[col_out] = df_out[col_out] + df_main[ft_]
    }
}

df_out
file_out = paste0(dir_fig, 'burden_count_table.tsv')
print(file_out)
write.table(df_out, file = file_out, sep = "\t", row.names = FALSE, quote = FALSE)

fid,iid,Phenotype,sex,MEAN_COVERAGE,Platform,rel,PC1,PC2,PC3,⋯,svs_lowpli_genes_exonic,trs_lowpli_genes_exonic,svs_pli_genes_intronic,trs_pli_genes_intronic,svs_pli_genes_exonic,trs_pli_genes_exonic,svs_fetal_brain_genes_intronic,trs_fetal_brain_genes_intronic,svs_fetal_brain_genes_exonic,trs_fetal_brain_genes_exonic
<chr>,<chr>,<int>,<fct>,<dbl>,<fct>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F0001,REACH000001,1,Male,10.15,ONT,C,0.008627126,-0.005607061,-0.003588309,⋯,12,30,50,118,1,14,38,66,1,8
F0026,REACH000026,1,Male,3.22,PB,C,0.095363540,-0.045229620,-0.019471550,⋯,16,0,62,0,1,0,30,0,1,0
F0058,REACH000058,1,Male,6.19,PB,C,0.002421759,-0.009216429,-0.007762021,⋯,8,0,41,1,2,1,28,2,1,0
F0065,REACH000065,1,Male,1.22,PB,C,0.252773800,0.107932900,0.199377900,⋯,11,0,65,0,3,0,52,0,3,0
F0078,REACH000086,1,Male,2.70,PB,C,0.044785250,0.038064880,0.054606420,⋯,13,0,43,2,2,0,38,2,4,0
F0078,REACH000089,1,Female,3.27,PB,C,0.049879690,0.047204300,0.069404520,⋯,11,0,40,2,1,0,30,1,1,0
F0081,REACH000092,1,Female,4.89,PB,C,-0.028510680,-0.001001894,-0.001920863,⋯,15,2,37,10,2,1,34,6,2,0
F0081,REACH000095,0,Female,5.12,PB,C,-0.030440730,-0.002736118,-0.006993734,⋯,23,2,45,7,2,1,38,5,4,0
F0083,REACH000097,1,Male,5.82,PB,C,0.095793110,0.052887850,0.100821100,⋯,23,2,60,13,0,1,46,8,2,0


[1] "R_clogit/for_publish_fin/burden_count_table.tsv"
